Here are the goals of this notebook:
- Identify significant peaks in power spectra
- Compare peak locations across groups
- Quantify peak characteristics (width, amplitude)

Requirements:
- A way to identify and characterize peaks in power spectra (frequency, width, amplitude, etc.)
- A way to compare peaks with task frequencies
- A way to compare peaks across groups
- A way to visualize peak analysis

Inputs:
- Power spectra data (my data is structured so that each frequency is a column and each row is a subject - region combination)

In [99]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from matplotlib.gridspec import GridSpec

In [ ]:
data_dir = Path('/Users/fdjim/Desktop/PDS_CODE/periodogram')
psd_hc_path = data_dir / 'psd_data_hc.csv'
psd_mdd_path = data_dir / 'psd_data_mdd.csv'
psd_sz_path = data_dir / 'psd_data_sz.csv'

# Read individual dataframes
df_hc = pd.read_csv(psd_hc_path)
df_mdd = pd.read_csv(psd_mdd_path)
df_sz = pd.read_csv(psd_sz_path)

# Add group column to each dataframe
df_hc['group'] = 'HC'
df_mdd['group'] = 'MDD'
df_sz['group'] = 'SZ'

# Combine into master dataframe
df_master = pd.concat([df_hc, df_mdd, df_sz], ignore_index=True)
# Reorder columns to put 'group' first
cols = ['group'] + [col for col in df_master.columns if col != 'group']
df_master = df_master[cols]

# Optional: Display first few rows to verify
df_master.head()

### Extract power spectrum

This means the frequencies and the power values, for a given subject - region combination.

In [9]:
def get_power_spectrum(df, roi, group):
    """
    Extract power spectrum for a specific ROI and group.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Your master dataframe
    roi : str
        Region of interest (e.g., 'motor', 'auditory')
    group : str
        Group identifier (e.g., 'HC', 'MDD', 'SZ')
        
    Returns:
    --------
    frequencies : numpy array
        Frequency values
    power : numpy array
        Average power values across subjects
    """
    # Filter data for specific ROI and group
    mask = (df['roi'] == roi) & (df['group'] == group)
    roi_data = df[mask]
    
    # Get frequency columns and convert to numeric values
    freq_cols = [col for col in df.columns if col.startswith('freq_')]
    frequencies = np.array([float(col.split('_')[1]) for col in freq_cols])
    
    # Calculate mean power across subjects
    power = roi_data[freq_cols].mean()
    
    return frequencies, power.values



In [10]:
# Let's test this with your motor region data
frequencies, power = get_power_spectrum(df_master, 'motor', 'HC')

### Peak detection
**Local Maxima**:
A peak is a point that's higher than its neighbors. However, not all local maxima are meaningful peaks. We need additional criteria.

**Prominence**:
This is the most important concept for distinguishing real peaks from noise. Think of prominence like this: if you're looking at a mountain peak, prominence is how far you need to descend before you can climb a higher peak. In your power spectra, this helps identify peaks that stand out from the background.

**Current Aim**:
- Figure out why the wavelet method is so shit (read documentation and learn more about it)
- Settle on a method and parameters, with peaks - put in PPT
- Quantify and learn about peak characteristics (width, amplitude and others)
- Compare peak charactersitics to task frequencies
- Compare peak characteristics across groups

In [19]:
class PeakDetector:
    """
    A class to detect and compare peaks using both direct and wavelet-based methods.
    
    This class implements two peak detection approaches:
    1. Direct peak detection (find_peaks)
    2. Wavelet-based peak detection (find_peaks_cwt)
    
    It's designed to work with power spectrum data where we have frequency
    and power arrays, making it particularly suitable for fMRI analysis.
    """
    
    def __init__(self, frequencies, power):
        """
        Initialize the peak detector with frequency and power data.
        
        Parameters:
        -----------
        frequencies : array-like
            Array of frequency values
        power : array-like
            Array of power values corresponding to frequencies
        """
        self.frequencies = np.array(frequencies)
        self.power = np.array(power)
        # Store detected peaks for later comparison
        self.direct_peaks = None
        self.wavelet_peaks = None
        
    def find_direct_peaks(self, prominence=0.1):
        """
        Find peaks using the direct method (find_peaks).
        
        Parameters:
        -----------
        prominence : float
            Minimum prominence for peak detection
            
        Returns:
        --------
        peak_indices : array
            Indices of detected peaks
        """
        peak_indices, properties = signal.find_peaks(self.power, 
                                                   prominence=prominence)
        self.direct_peaks = peak_indices
        return peak_indices
    
    def find_wavelet_peaks(self, widths=None, min_snr=1.5):
        """
        Find peaks using the wavelet method (find_peaks_cwt).
        
        Parameters:
        -----------
        widths : array-like, optional
            Wavelet widths to use. If None, automatically determined
        min_snr : float
            Minimum signal-to-noise ratio for peak detection
            
        Returns:
        --------
        peak_indices : array
            Indices of detected peaks
        """
        if widths is None:
            # Automatically determine reasonable width range based on data length
            widths = np.arange(1, min(20, len(self.power) // 10))
            
        peak_indices = signal.find_peaks_cwt(self.power, 
                                           widths=widths,
                                           min_snr=min_snr)
        self.wavelet_peaks = peak_indices
        return peak_indices
    
    def plot_comparison(self, title=None):
        """
        Plot the power spectrum with peaks detected by both methods.
        
        Parameters:
        -----------
        title : str, optional
            Title for the plot
        """
        plt.figure(figsize=(12, 6))
        
        # Plot original power spectrum
        plt.plot(self.frequencies, self.power, 'b-', 
                label='Power Spectrum', alpha=0.7)
        
        # Plot peaks found by direct method if available
        if self.direct_peaks is not None:
            plt.plot(self.frequencies[self.direct_peaks], 
                    self.power[self.direct_peaks], 
                    'ro', label='Direct Peaks')
            
        # Plot peaks found by wavelet method if available
        if self.wavelet_peaks is not None:
            plt.plot(self.frequencies[self.wavelet_peaks], 
                    self.power[self.wavelet_peaks], 
                    'gx', markersize=10, label='Wavelet Peaks')
            
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power')
        plt.title(title or 'Peak Detection Comparison')
        plt.yscale('log')
        plt.xlim(0.01, 0.24)
        plt.ylim(10**0, 10**2)
        plt.xticks(np.arange(0.01, 0.25, 0.01))
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
# Example usage with HC motor region
frequencies, power = get_power_spectrum(df_master, 'motor', 'HC')

# Create peak detector
detector = PeakDetector(frequencies, power)

# Find peaks using both methods
detector.find_direct_peaks(prominence=0.1)
detector.find_wavelet_peaks(min_snr=1.5)

# Plot comparison
detector.plot_comparison(title='Motor Region Peak Detection Comparison')

In [126]:
class ParameterExplorer:
    """
    A class to systematically explore parameter spaces in signal processing.
    
    This represents a modern computational approach to experimentation,
    where we automate the process of testing different parameters
    and collecting results.
    This version includes visualization capabilities from PeakDetector.

    """
    def __init__(self, df_master=None, frequencies=None, power=None):
        """
        Initialize with either a master dataframe or specific frequency/power data.
        
        Parameters:
        -----------
        df_master : pandas DataFrame, optional
            Master dataframe containing all regions/groups
        frequencies : array-like, optional
            Array of frequency values (if not using df_master)
        power : array-like, optional
            Array of power values (if not using df_master)
        """
        self.df_master = df_master
        self.frequencies = frequencies
        self.power = power
        self.results = {}
        
        if df_master is None and (frequencies is None or power is None):
            raise ValueError("Must provide either df_master or both frequencies and power")
            
        if frequencies is not None:
            self.detector = PeakDetector(frequencies, power)
        
    def explore_direct_peaks(self, prominences):
        """
        Systematically test different prominence values.
        
        Think of this like running multiple trials in a lab experiment,
        but automated and perfectly reproducible.
        """
        results = []
        for prominence in prominences:
            detector = PeakDetector(self.frequencies, self.power)
            peaks = detector.find_direct_peaks(prominence=prominence)
            
            # Store results of this "experiment"
            results.append({
                'prominence': prominence,
                'n_peaks': len(peaks),
                'peak_locations': peaks,
                'peak_powers': self.power[peaks]
            })
            
        self.results['direct_peaks'] = results
        return results
    
    def explore_wavelet_peaks(self, width_ranges, min_snrs):
        """
        Systematically test combinations of wavelet parameters.
        
        This is like running a factorial experiment design,
        testing all combinations of parameters.
        """
        results = []
        for widths in width_ranges:
            for min_snr in min_snrs:
                detector = PeakDetector(self.frequencies, self.power)
                peaks = detector.find_wavelet_peaks(
                    widths=widths, min_snr=min_snr)
                
                results.append({
                    'width_range': f"{widths[0]}-{widths[-1]}",
                    'min_snr': min_snr,
                    'n_peaks': len(peaks),
                    'peak_locations': peaks,
                    'peak_powers': self.power[peaks]
                })
                
        self.results['wavelet_peaks'] = results
        return results
    
    def visualize_parameter_effects(self):
        """
        Create a comprehensive visualization of how parameters
        affect peak detection results.
        """
        if 'direct_peaks' in self.results:
            # Plot how number of peaks varies with prominence
            prominences = [r['prominence'] for r in self.results['direct_peaks']]
            n_peaks = [r['n_peaks'] for r in self.results['direct_peaks']]
            
            plt.figure(figsize=(10, 6))
            plt.plot(prominences, n_peaks, 'o-')
            plt.xlabel('Prominence')
            plt.ylabel('Number of Peaks Detected')
            plt.title('Effect of Prominence on Peak Detection')
            plt.show()

    def plot_experiment_comparison(self, experiment_type, experiment_index):
        """
        Plot comparison for a specific experiment using PeakDetector's
        visualization capability.
        
        Parameters:
        -----------
        experiment_type : str
            Either 'direct_peaks' or 'wavelet_peaks'
        experiment_index : int
            Index of the experiment to visualize
        """
        if experiment_type not in self.results:
            raise ValueError(f"No results found for {experiment_type}")
            
        # Get the experiment results
        experiment = self.results[experiment_type][experiment_index]
        
        # Update the detector with these peaks
        if experiment_type == 'direct_peaks':
            self.detector.direct_peaks = experiment['peak_locations']
            title = f"Direct Peak Detection (prominence={experiment['prominence']})"
        else:
            self.detector.wavelet_peaks = experiment['peak_locations']
            title = (f"Wavelet Peak Detection "
                    f"(width range={experiment['width_range']}, "
                    f"min_snr={experiment['min_snr']})")
            
        # Use the detector's plotting function
        self.detector.plot_comparison(title=title)

    def explore_peaks_across_regions(self, regions=None, groups=None, prominence_range=None):
        """
        Systematically explore peak detection across specified brain regions and groups.
        
        Parameters:
        -----------
        regions : list of str, optional
            List of regions to analyze. Default: ['motor', 'visual', 'dmn', 'fpn']
        groups : list of str, optional
            List of groups to analyze. Default: ['HC']
        prominence_range : tuple or array-like, optional
            Either (start, stop, num_points) for logspace or array of prominence values
            Default: np.logspace(-1, 0.5, 20)
            
        Returns:
        --------
        dict
            Nested dictionary of results by region, group, and prominence value
        """
        if self.df_master is None:
            raise ValueError("df_master is required for cross-region analysis")
            
        # Set defaults
        regions = regions or ['motor', 'visual', 'dmn', 'fpn']
        groups = groups or ['HC']
        
        if prominence_range is None:
            prominences = np.logspace(-1, 0.5, 20)
        elif isinstance(prominence_range, (tuple, list)) and len(prominence_range) == 3:
            start, stop, num = prominence_range
            prominences = np.logspace(start, stop, num)
        else:
            prominences = np.array(prominence_range)
        
        results_by_region = {}
        for region in regions:
            results_by_region[region] = {}
            for group in groups:
                # Get power spectrum for this region/group
                frequencies, power = get_power_spectrum(self.df_master, region, group)
                
                # Create detector for this spectrum
                detector = PeakDetector(frequencies, power)
                
                # Test different prominence values
                region_results = []
                for prominence in prominences:
                    peaks = detector.find_direct_peaks(prominence=prominence)
                    region_results.append({
                        'prominence': prominence,
                        'n_peaks': len(peaks),
                        'peak_locations': peaks,
                        'peak_frequencies': frequencies[peaks],
                        'peak_powers': power[peaks],
                        # 'peak_fwhm': [calculate_fwhm(frequencies, power, peak) 
                        #             for peak in peaks]
                    })
                results_by_region[region][group] = region_results
        
        self.results['cross_region'] = results_by_region
        return results_by_region    
   
    def visualize_peaks_across_regions(self, regions=None, groups=None, 
                                    prominence_indices=None, figsize=None, 
                                    save_path=None, max_plots=12):
        """
        Visualize PSD plots with detected peaks for specified regions, groups, and prominence values.
        
        Parameters:
        -----------
        regions : list of str, optional
            List of regions to plot. Defaults to all regions in results
        groups : list of str, optional
            List of groups to plot. Defaults to all groups in results
        prominence_indices : list of int, optional
            Indices of prominence values to plot. If None, plots evenly spaced values
        figsize : tuple, optional
            Figure size (width, height). Default: auto-calculated
        save_path : str, optional
            Path to save the figure. If None, figure is displayed only
        max_plots : int, optional
            Maximum number of prominence values to plot to avoid overcrowding
            
        Returns:
        --------
        fig : matplotlib.figure.Figure
            The created figure object
        """
        ### Error Handling ###
        if 'cross_region' not in self.results:
            raise ValueError("No cross-region results found. Run explore_peaks_across_regions first.")
        
        ### Get Available Regions and Groups ###
        available_regions = list(self.results['cross_region'].keys())
        regions = regions or available_regions
        regions = [r for r in regions if r in available_regions]
        available_groups = list(self.results['cross_region'][regions[0]].keys())
        groups = groups or available_groups
        
        ### Get Prominence Values, Indices and Plot ###
        all_results = self.results['cross_region'][regions[0]][groups[0]]
        n_prominences = len(all_results)      
        if prominence_indices is None:
            # Select evenly spaced indices, but no more than max_plots
            step = max(1, n_prominences // max_plots)
            prominence_indices = list(range(0, n_prominences, step))
            if len(prominence_indices) > max_plots:
                prominence_indices = prominence_indices[:max_plots]

        ### Figure Setup ###
        # Calculate grid layout
        plots_per_prominence = len(regions) * len(groups)
        n_prominences_to_plot = len(prominence_indices)
        
        # Adjust figure size to accommodate legend
        if figsize is None:
            figsize = (15, 4 * n_prominences_to_plot + 1)  # Add extra space for legend
        fig = plt.figure(figsize=figsize)
        
        # Create grid specification for both legend and plots
        gs = GridSpec(n_prominences_to_plot + 1, plots_per_prominence,
                     height_ratios=[0.4] + [1] * n_prominences_to_plot,
                 hspace=0.5,
                 wspace=0.3)

        # Create legend axis spanning all columns in first row
        legend_ax = fig.add_subplot(gs[0, :])
        legend_ax.axis('off')
        
        # Create legend
        legend_elements = [
            plt.Line2D([0], [0], color='b', label='PSD'),
            plt.Line2D([0], [0], color='r', marker='o', label='Peaks', linestyle='None')
        ]
        
        ### Task Frequencies ###
        task_freqs = {
            'Single Question': [0.125, 0.25, 0.375],  # 8s period
            'Question Block': [0.0313, 0.0626, 0.0939],  # 32s period
            'Resting State': [0.05, 0.10, 0.15],  # 20s period
            'Full Block': [0.0192, 0.0384, 0.0576]  # 52s period
        }

        # Define colors for each task type
        task_colors = {
            'Single Question': '#FFA500',   # Orange
            'Question Block': '#4CAF50',    # Green
            'Resting State': '#9C27B0',     # Purple
            'Full Block': '#795548'         # Brown
        }
        # Add task frequency lines to legend elements
        for task_name, frequencies in task_freqs.items():
            for i, freq in enumerate(frequencies):
                linestyle = '-' if i == 0 else '--'
                alpha = 0.8 if i == 0 else 0.5
                label = f'{task_name}\n{"Fundamental" if i==0 else f"Harmonic {i+1}"}\n({freq:.3f} Hz)'
                legend_elements.append(
                plt.Line2D([0], [0], color=task_colors[task_name], 
                          linestyle=linestyle, alpha=alpha,
                          label=label)
            )

        # Adjust legend formatting
        legend_ax.legend(handles=legend_elements, 
                        loc='center', 
                        ncol=10,  # Adjust number of columns as needed
                        bbox_to_anchor=(0.5, 0.5),
                        fontsize=8,
                        labelspacing=1)  # Increase spacing between legend entries
                        
        for prom_idx, prominence_index in enumerate(prominence_indices):
            prominence_value = all_results[prominence_index]['prominence']
            for r_idx, region in enumerate(regions):
                for g_idx, group in enumerate(groups):
                    plot_num = r_idx * len(groups) + g_idx + 1
                    ax = fig.add_subplot(gs[prom_idx + 1, r_idx * len(groups) + g_idx])

                    # Get data and peaks for this prominence
                    frequencies, power = get_power_spectrum(self.df_master, region, group)
                    results = self.results['cross_region'][region][group][prominence_index]
                    peaks = results['peak_locations']
                    
                    # Plot PSD and peaks without labels
                    ax.plot(frequencies, power, 'b-')  # Removed label
                    ax.plot(frequencies[peaks], power[peaks], 'ro')  # Removed label
                    
                    # Add peak annotations
                    for peak_idx in peaks:
                        ax.annotate(f'{frequencies[peak_idx]:.3f}Hz',
                                xy=(frequencies[peak_idx], power[peak_idx]),
                                xytext=(5, 5), textcoords='offset points',
                                fontsize=8)
                    
                    # Add vertical lines for task frequencies and harmonics without labels
                    for task_name, frequencies in task_freqs.items():
                        for i, freq in enumerate(frequencies):
                            linestyle = '-' if i == 0 else '--'
                            alpha = 0.8 if i == 0 else 0.5
                            ax.axvline(x=freq, color=task_colors[task_name], 
                                    linestyle=linestyle, alpha=alpha)  # Removed label

                    # Customize plot
                    ax.set_title(f'{region.upper()} - {group}\nprom={prominence_value:.3f}')
                    if r_idx == len(regions)-1:
                        ax.set_xlabel('Frequency (Hz)')
                    if g_idx == 0:
                        ax.set_ylabel('Power')
                    ax.set_yscale('log')
                    ax.set_ylim(10**0, 10**2)
                    ax.set_xlim(0.01, 0.24)
                    ax.grid(True, alpha=0.3)

        # Adjust figure size to reduce white space
        if figsize is None:
            figsize = (15, 3 * n_prominences_to_plot + 1)  # Reduced height multiplier
        fig = plt.figure(figsize=figsize)

        # Add tight_layout with adjusted rect parameter
        plt.tight_layout(rect=[0, 0.02, 1, 0.98])  # Adjusted rect parameters to reduce white space

        # Adjust suptitle position
        fig.suptitle(f'Peak Detection Analysis Across Prominence Values\n' +
                    f'Regions: {", ".join(regions)} | Groups: {", ".join(groups)}',
                    fontsize=12, y=0.99)  # Adjusted y position                
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
            
        return fig

    def plot_all_experiments(self):
        """
        Create a grid of comparison plots for all experiments.
        This gives us a comprehensive view of how different parameters
        affect peak detection.
        """
        # Calculate grid dimensions
        n_direct = len(self.results.get('direct_peaks', []))
        n_wavelet = len(self.results.get('wavelet_peaks', []))
        n_total = n_direct + n_wavelet
        
        # Calculate reasonable grid dimensions
        n_cols = min(3, n_total)  # Maximum 3 columns
        n_rows = (n_total + n_cols - 1) // n_cols
        
        # Create subplot grid
        fig = plt.figure(figsize=(6*n_cols, 4*n_rows))
        
        # Plot direct peak experiments
        for i, exp in enumerate(self.results.get('direct_peaks', [])):
            plt.subplot(n_rows, n_cols, i + 1)
            self.detector.direct_peaks = exp['peak_locations']
            self.detector.wavelet_peaks = None
            self._plot_single_comparison(
                f"Prominence = {exp['prominence']:.3f}")
            
        # Plot wavelet peak experiments
        offset = len(self.results.get('direct_peaks', []))
        for i, exp in enumerate(self.results.get('wavelet_peaks', [])):
            plt.subplot(n_rows, n_cols, i + offset + 1)
            self.detector.direct_peaks = None
            self.detector.wavelet_peaks = exp['peak_locations']
            self._plot_single_comparison(
                f"Width={exp['width_range']}\nSNR={exp['min_snr']}")
        
        plt.tight_layout()
        # plt.show()
        
    def _plot_single_comparison(self, title):
        """
        Helper method for plotting a single comparison subplot.
        Reuses code from PeakDetector but adapts it for subplots.
        """
        # Plot original power spectrum
        plt.plot(self.frequencies, self.power, 'b-', 
                label='Power Spectrum', alpha=0.7)
        
        # Plot peaks found by direct method if available
        if self.detector.direct_peaks is not None:
            plt.plot(self.frequencies[self.detector.direct_peaks], 
                    self.power[self.detector.direct_peaks], 
                    'ro', label='Direct Peaks')
            
        # Plot peaks found by wavelet method if available
        if self.detector.wavelet_peaks is not None:
            plt.plot(self.frequencies[self.detector.wavelet_peaks], 
                    self.power[self.detector.wavelet_peaks], 
                    'gx', markersize=10, label='Wavelet Peaks')

        plt.xlim(0.01, 0.24)
        plt.ylim(10**0, 10**2)
        plt.xticks(np.arange(0.01, 0.25, 0.02))
  
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power')
        plt.title(title)
        plt.yscale('log')
        plt.legend()
        plt.grid(True)

In [ ]:
# Basic usage - will plot all regions and groups
prominence_range = np.arange(1.1, 2.3, 0.1)
explorer = ParameterExplorer(df_master=df_master)
explorer.explore_peaks_across_regions(groups=['HC', 'MDD', 'SZ'], regions=['motor'], prominence_range=prominence_range)
explorer.visualize_peaks_across_regions();

# Plot specific prominence indices
# explorer.visualize_peaks_across_regions(
#     regions=['motor', 'visual'],
#     prominence_indices=[0, 5, 10, 15]  # Plot results for these specific prominence values
# )


In [ ]:
# Example usage
explorer = ParameterExplorer(frequencies, power)

# Run experiments
prominences = [0.1, 0.2, 0.5]
width_ranges = [np.arange(1, 10), np.arange(1, 20)]
min_snrs = [1.5]

explorer.explore_direct_peaks(prominences)
explorer.explore_wavelet_peaks(width_ranges, min_snrs)

# Plot all experiments in a grid
explorer.plot_all_experiments()

# Or plot a single experiment
explorer.plot_experiment_comparison('direct_peaks', 2)



In [ ]:
# Prominence Exploration

prominence_explorer = ParameterExplorer(frequencies, power)
prominences = np.arange(0.1, 2, 0.1)
prominence_explorer.explore_direct_peaks(prominences)
prominence_explorer.plot_all_experiments()
prominence_explorer.visualize_parameter_effects()

In [ ]:
# Specific Peak Detection
# FWHM Calculation

prominence_explorer = ParameterExplorer(frequencies, power)
prominences = np.arange(0.5, 2.5, 0.1)
prominence_explorer.explore_direct_peaks(prominences)
prominence_explorer.plot_all_experiments()
prominence_explorer.visualize_parameter_effects()

In [ ]:
def calculate_fwhm(frequencies, power, peak_idx, window_size=10):
    """
    Calculate FWHM for a peak using interpolation for better accuracy.
    
    Parameters:
    -----------
    frequencies : array
        Array of frequency values
    power : array
        Array of power values
    peak_idx : int
        Index of the peak
    window_size : int
        How many points to look on each side of the peak
        
    Returns:
    --------
    fwhm : float
        Full Width at Half Maximum
    """
    # Get a window around the peak
    start_idx = max(0, peak_idx - window_size)
    end_idx = min(len(power), peak_idx + window_size)
    
    # Get the data in our window
    freq_window = frequencies[start_idx:end_idx]
    power_window = power[start_idx:end_idx]
    
    # Find half maximum value
    peak_power = power[peak_idx]
    half_power = peak_power / 2
    
    # Use interpolation to find more precise crossing points
    from scipy import interpolate
    
    # Create interpolation function
    f = interpolate.interp1d(freq_window, power_window - half_power, 
                            kind='cubic', bounds_error=False)
    
    # Create dense frequency array for interpolation
    freq_dense = np.linspace(freq_window[0], freq_window[-1], 1000)
    
    # Find zero crossings (where power = half_power)
    power_dense = f(freq_dense)
    zero_crossings = np.where(np.diff(np.signbit(power_dense)))[0]
    
    if len(zero_crossings) >= 2:
        # Get the two crossings closest to the peak
        fwhm = freq_dense[zero_crossings[1]] - freq_dense[zero_crossings[0]]
        return fwhm
    else:
        return None

# Now we can update your results structure:
results.append({
    'prominence': prominence,
    'n_peaks': len(peaks),
    'peak_locations': peaks,
    'peak_powers': self.power[peaks],
    'peak_fwhm': [calculate_fwhm(self.frequencies, self.power, peak) 
                  for peak in peaks]
})